In [1]:
import os
import google.generativeai as genai

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-pro")

## SQL 문제 테스트

In [2]:
questions = {1:"예상 배송일보다 실제 배송을 늦게 받은 고객 중에서 가장 많은 주문을 한 고객의 ID와 총 주문 수를 조회하세요. \
- 결과 컬럼: customer_id, total_orders",
2:"payments 테이블에서 각 결제 방식(payment_type)별 결제 금액의 합계와 해당 결제 방식이 전체 결제 금액에서 차지하는 비율을 계산하세요.\
- 결과 컬럼: payment_type, total_payment_value, payment_percentage",
3:"배송된(delivered) 주문을 기준으로 고유 고객 수, 총 주문 수, 총 결제 금액, 그리고 고객 1명당 평균 결제액을 계산하세요.\
- 결과 컬럼: cnt_users, cnt_orders, sum_payment, arppu",
4:"동일한 결제수단(payment_type)에서 이루어진 다른 결제의 평균 금액보다 높은 결제들 중에서, 해당 결제가 해당 결제수단의 총 결제 금액 대비 20% 이상을 차지하는 주문을 조회하세요.\
payment_ratio는 결제 금액이 총 결제 금액에서 차지하는 비율을 소수점 둘째 자리까지 계산해주세요.\
- 결과 컬럼: order_id, payment_type, payment_value, payment_ratio",
5:"orders 테이블에서 월별(년-월) 주문 건수를 계산하되, **주문이 없는 달도 0건으로 포함**하고, 지난달 대비 주문 건수 증감율(growth_rate)을 계산하세요.\
결과는 년-월(month) 순서대로 정렬하며, 증감율은 소수점 둘째 자리까지 반올림 해주세요.\
- 결과 컬럼: month, cnt_orders, growth_rate",
6:"각 결제 방식(payment_type)별 결제 금액의 평균 ± 3 표준편차(standard deviation)를 기준으로 이상치를 **`‘Yes’/’No’`**로 탐지하세요. 결제 금액이 이 범위를 벗어나면 이상치로 간주합니다.\
결제 금액이 큰 순으로 정렬해주세요.\
- 결과 컬럼: order_id, payment_type, payment_value, is_outlier"}

In [4]:
correct_answer = {1:"""SELECT 
    o.customer_id,
    COUNT(o.order_id) AS total_orders
FROM orders o
WHERE order_estimated_delivery_date < order_delivered_customer_date
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;""",
                  2:"""SELECT 
    payment_type,
    SUM(payment_value) AS total_payment_value,
    ROUND(SUM(payment_value) * 100.0 / (SELECT SUM(payment_value) FROM payments), 2) AS payment_percentage
FROM payments
GROUP BY 1
ORDER BY 2 DESC;""",
                  3:"""SELECT 
    COUNT(DISTINCT o.customer_id) AS cnt_users,
    COUNT(o.order_id) AS cnt_orders,
    SUM(p.payment_value) AS sum_payment,
    CASE
        WHEN COUNT(DISTINCT o.customer_id) = 0 THEN 0
        ELSE SUM(p.payment_value) / COUNT(DISTINCT o.customer_id)
    END AS arppu 
FROM orders o
INNER JOIN payments p ON o.order_id = p.order_id
WHERE o.order_status = 'delivered';""",
                  4:"""WITH payment_summary AS (
    SELECT 
        payment_type,
        AVG(payment_value) AS avg_payment_value,
        SUM(payment_value) AS total_payment_value
    FROM payments
    GROUP BY payment_type
)
SELECT 
    p.order_id,
    p.payment_type,
    p.payment_value,
    ROUND(p.payment_value / ps.total_payment_value * 100, 2) AS payment_ratio
FROM payments p
JOIN payment_summary ps ON p.payment_type = ps.payment_type
WHERE p.payment_value > ps.avg_payment_value -- 조건1 체크!
AND p.payment_value / ps.total_payment_value >= 0.2; -- 조건2 체크!""",
                  5:"""WITH RECURSIVE all_months AS (
    SELECT DATE_FORMAT(MIN(order_purchase_timestamp), '%Y-%m') AS months
    FROM orders
    UNION ALL
    SELECT DATE_FORMAT(DATE_ADD(CONCAT(months, '-01'), INTERVAL 1 MONTH), '%Y-%m')
    FROM all_months
    WHERE DATE_ADD(CONCAT(months, '-01'), INTERVAL 1 MONTH) <= (
        SELECT DATE_FORMAT(MAX(order_purchase_timestamp), '%Y-%m-01') FROM orders
    )
),
monthly_orders AS (
    SELECT 
        m.months,
        COALESCE(COUNT(o.order_id), 0) AS cur_orders
    FROM all_months m
    LEFT JOIN orders o ON DATE_FORMAT(o.order_purchase_timestamp, '%Y-%m') = m.months
    GROUP BY m.months
)
SELECT 
    months,
    cur_orders,
    prev_orders,
    ROUND(
        CASE 
            WHEN prev_orders IS NULL THEN NULL
            WHEN prev_orders = 0 THEN cur_orders * 100
            ELSE (cur_orders - prev_orders) / prev_orders * 100
        END, 
        2
    ) AS growth_rate
FROM (
    SELECT 
        months,
        cur_orders,
        LAG(cur_orders) OVER (ORDER BY months) AS prev_orders
    FROM monthly_orders
) sub
ORDER BY 1;""",
                  6:"""SELECT
	p.order_id,
	p.payment_type,
	p.payment_value,
	CASE
		WHEN p.payment_value < ps.avg_payment - 3 * ps.stddev_payment
		OR p.payment_value > ps.avg_payment + 3 * ps.stddev_payment THEN 'Yes'
		ELSE 'No'
	END AS is_outlier
FROM payments p
JOIN (
	SELECT
		payment_type,
		AVG(payment_value) AS avg_payment,
		STDDEV(payment_value) AS stddev_payment
	FROM payments
	GROUP BY 1) AS ps 
ON p.payment_type = ps.payment_type
ORDER BY 3 desc;
                  """}

'예상 배송일보다 실제 배송을 늦게 받은 고객 중에서 가장 많은 주문을 한 고객의 ID와 총 주문 수를 조회하세요. - 결과 컬럼: customer_id, total_orders'

In [17]:
question = """
문제1. 짝수 판별하기
1. 출제의도
    * 조건문을 사용하고 출력함수를 사용할 수 있다.
2. 배경
    * 당신은 간단한 조건문을 이용해서 숫자를 분별하려고합니다.
3. 요구사항
    * 무작위 정수 n 이 주어집니다.
    * 변수 n 을 가지고 짝수인지 홀수인지 0인지 출력하는 파이썬 코드를 만드세요. 함수형이 아니여도 괜찮습니다.
    * 짝수는 2로 나누었을때 나누어 떨어지는 숫자입니다.
    * 홀수는 2로 나누었을때 나머지가 1이 남는 숫자 입니다.
    * n = 3, n= 2, n= 0 인 케이스 모두 3가지를 채점할 예정입니다.
4. Skeleton code
n=3
5. 출력 결과
	‘’’
	이 숫자는 홀수입니다
	‘’’
"""

In [18]:
correct_answer = """
n = 3  # 여기 값을 2 또는 0으로도 바꿔서 테스트 가능

if n == 0:
    print("이 숫자는 0입니다")
elif n % 2 == 0:
    print("이 숫자는 짝수입니다")
else:
    print("이 숫자는 홀수입니다")
"""

In [19]:
student_answer = """
def check_even_odd(n):
    if n % 2 == 0:
        print("이 숫자는 짝수입니다")
    else:
        print("이 숫자는 홀수입니다")

check_even_odd(3)
"""

In [20]:
question2 = """
제품 재고 관리**

1. **출제의도**
    - 리스트의 인덱스를 이해하고 원소에 접근할 수 있다.
    - 반복문과 리스트 자료형을 이용하여 기능을 구현할 수 있다.
    - print 내장함수에 변수를 포함시켜 출력할 수 있다.
2. **배경**
    - 당신은 소매점에서 근무하며 제품의 재고 관리를 담당하고 있습니다.  재고가 부족하거나 과잉인 제품을 식별하는 것이 중요합니다. 주어진 제품의 재고 데이터를 확인하여, 재고의 상태를 나타내는 파이썬 코드를 만드세요.
3. **요구사항**
    - inventory_list 리스트의 각 요소는 현재 재고비율 형식의 리스트입니다.
        - 예: [90, 60, 80]
        - 해석: 1번째 아이템은 90%, 2번째 아이템은 60%, 3번째 아이템은 80%의 재고가 남아있습니다.
    - 기준치 value를 받습니다. 기준치 미만이면 재고가 부족한 것으로, 기준치 초과면 재고가 과잉인 것으로 기준치와 같다면 적절한 것으로 간주합니다
    - 각 제품의 재고량이 주어진 기준치보다 많은지 적은지를 적절한지 판단합니다.
    - ex) 기준치를 80로 설정한다면 첫 번째 아이템은 **과잉**, 두 번째 아이템은 **부족**입니다, 세번째 아이템은 적절으로 출력하는 기능을 만듭니다.

1. **Skeleton 코드**
    
    
python
    inventory_list = [90,60,80]
    value = 80
    '''
    여기에 들어갈 코드를 작성하세요
    '''

    
2. **출력 결과**
'''
현재 기준 값은 80% 입니다.
1 번째 아이템 재고는 90% 로 과잉입니다.
2 번째 아이템 재고는 60% 로 부족합니다.
3 번째 아이템 재고는 80% 로 적절합니다.
'''
"""

correct_answer2 = """
inventory_list = [90, 60, 80]
value = 80

print(f"현재 기준 값은 {value}% 입니다.")

for i in range(len(inventory_list)):
    stock = inventory_list[i]
    if stock > value:
        status = "과잉"
    elif stock < value:
        status = "부족"
    else:
        status = "적절"
    print(f"{i+1} 번째 아이템 재고는 {stock}% 로 {status}입니다.")
"""

student_answer2 = """
inventory_list = [90, 60, 80]
value = 80

print(f"현재 기준 값은 {value}% 입니다.")

for idx, stock in enumerate(inventory_list, start=1):
    status = (
        "과잉" if stock > value else
        "부족" if stock < value else
        "적절"
    )
    print(f"{idx} 번째 아이템 재고는 {stock}% 로 {status}입니다.")

"""

In [21]:
question3 = """
이메일 유효성 검사

1. 출제의도
    - 문자형 자료형의 메소드를 사용할 수 있다.
    - 조건문,반복문, 함수형을 완벽하게 이해하여 기능을 구현할 수 있다.
2. **배경**

당신은 고객 데이터를 수집하고 정리하는 과정에 있습니다. 확인해보니 회원가입 시 이메일 유효성 검사가 기능으로 구현되어있지 않음을 확인했습니다. 이를 보완하기 위하여 이메일 유효성 검사 기능을 만들려고 합니다. 

3. **요구사항**
- 이메일은 ID와 골뱅이, 도메인 3가지 요소로 구성 되어있습니다.
    - 이메일은 하나의 @ 기호를 포함해야하며, @ 기준으로 **ID**와 **도메인**으로 나눠집니다.
    - **ID**는 어떠한 문자든 숫자든 허용됩니다. 최소한 1자리가 존재 해야합니다.
    - **도메인** 역시 어떠한 문자나 숫자를 허용합니다. 단, 접미사는 .com , .co.kr 만 허용합니다.(문제풀이 편의성)
        
        ![https://blog.mailplug.com/1257](https://prod-files-secure.s3.us-west-amazonaws.com/83c75a39-3aba-4ba4-a792-7aefe4b07895/c13b910a-15db-436f-8578-6adc9b5cc90e/image.png)
- 이메일 주소 리스트 email_list 를 입력받아 각 이메일의 유효성을 검사하는 함수 check_email 함수를 완성합니다.
- 유효한 이메일인 경우 “유효한 이메일입니다” 라고 출력합니다.
- 유효하지 않은 이메일인 경우 “유효하지 않은 이메일입니다.” 라고 출력하며 그 이유를 함께 출력합니다.
    - ex) “ID가 누락”, “@ 가 누락”, “적절하지 않은 도메인”
4. **Skeleton 코드**
email_list = [
    "example@example.com",  # 유효한 이메일
    "yangbujang@email.co.kr",  # 유효한 이메일
    "iamhungry.com",  # 틀린 이메일 -> @ 누락
    "@da.com",  # 틀린 이메일 -> ID 누락
    "daislove@isnit",  # 틀린 이메일 -> 도메인 잘못됨
]

def check_email(email_list):
		'''
		여기에 들어갈 코드를 작성하세요
		'''
check_email(email_list)
5. 출력결과
'''
유효한 이메일입니다: 'example@example.com'
유효한 이메일입니다: 'yangbujang@email.co.kr'
유효하지 않은 이메일입니다: 'iamhungry.com' 이유: '@가 누락'
유효하지 않은 이메일입니다: '@da.com' 이유: 'ID가 누락'
유효하지 않은 이메일입니다: 'daislove@isnit'  이유: '적절하지 않은 도메인'
'''
"""

correct_answer3="""
email_list = [
    "example@example.com",
    "yangbujang@email.co.kr",
    "iamhungry.com",
    "@da.com",
    "daislove@isnit"
]

def check_email(email_list):
    for email in email_list:
        # 1. '@' 기호 포함 여부 확인
        if '@' not in email:
            print(f"유효하지 않은 이메일입니다: '{email}' 이유: '@가 누락'")
            continue

        # 2. ID, 도메인 분리
        parts = email.split('@')
        if len(parts) != 2:
            print(f"유효하지 않은 이메일입니다: '{email}' 이유: '@가 두 개 이상이거나 형식이 잘못됨'")
            continue

        id_part, domain_part = parts

        # 3. ID 확인
        if not id_part:
            print(f"유효하지 않은 이메일입니다: '{email}' 이유: 'ID가 누락'")
            continue

        # 4. 도메인 확인
        if not (domain_part.endswith(".com") or domain_part.endswith(".co.kr")):
            print(f"유효하지 않은 이메일입니다: '{email}' 이유: '적절하지 않은 도메인'")
            continue

        # 5. 유효한 경우
        print(f"유효한 이메일입니다: '{email}'")

check_email(email_list)
"""

student_answer3="""
def check_email(email_list):
    for email in email_list:
        if '@' in email:
            print(f"유효한 이메일입니다: '{email}'")
        else:
            print(f"유효하지 않은 이메일입니다: '{email}' 이유: '@가 누락'")
"""

In [22]:
def grade_answer(question, model_answer, student_answer):
    prompt = f"""
다음은 Python 과제입니다.

📌 문제:
{question}

✅ 모범 답안:
{correct_answer}

❌ 학생 답안:
{student_answer}

아래 기준에 따라 학생의 이해도를 평가하고, 간단한 피드백을 작성해주세요.

1. 학생의 답안을 '상', '중', '하' 중 하나로 평가해주세요.
    - 상: 문제를 정확히 이해하고 요구사항에 맞는 답안을 작성함
    - 중: 일부 요구사항을 충족했지만 불완전하거나 오류가 있음
    - 하: 문제를 거의 이해하지 못했거나 잘못된 접근을 함

2. 왜 그렇게 평가했는지 간단하게 2~3문장으로 설명해주세요.

결과는 다음 형식으로 출력해주세요:

[이해도 평가] 중  
[피드백] 학생은 문제의 요구사항 중 일부를 이해했으나, 0에 대한 예외 처리가 빠져 있어 정확한 조건 분기가 어렵습니다.
"""
    response = model.generate_content(prompt)
    return response.text


In [23]:
%%time
print(grade_answer(question, correct_answer, student_answer))

[이해도 평가] 중
[피드백] 학생은 짝수와 홀수를 판별하는 조건문을 작성했지만, n이 0인 경우를 처리하지 못했습니다. 문제에서 n=0인 경우를 명시적으로 요구했으므로, 이 부분을 추가해야 완전한 답안이 됩니다.

CPU times: total: 93.8 ms
Wall time: 2.17 s
